In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import coremltools as cmt
from sklearn.model_selection import train_test_split

# Data Preparation

In [ ]:
data = pd.read_csv('rgb_data.csv')

colors = list(set(data.label.values))
colorDict = {}
    
for i in range(len(colors)):
    color = colors[i]
    colorDict[i] = color
    data = data.replace(to_replace=color, value=i)

Y = []
for item in data['label']:
    a = [0,0,0,0,0,0,0,0,0,0,0]
    a[item] = 1
    Y.append(a)

y = pd.DataFrame(Y)
X = data[['red','green','blue']]
x_train, x_test, y_train, y_test = train_test_split(X,y, test_size= 0.20, random_state=27)

# Create Model

In [ ]:
model = keras.Sequential()

input_layer = layers.Dense(3, activation='relu')
hidden_layer1 = layers.Dense(100, activation='relu')
hidden_layer2 = layers.Dense(100, activation='relu')
hidden_layer3 = layers.Dense(100, activation='relu')
output_layer = layers.Dense(11, activation='softmax')

for layer in [input_layer, hidden_layer1, hidden_layer2, hidden_layer3, output_layer]:
    model.add(layer)
    
model.compile(optimizer='adam',loss='categorical_crossentropy')

history = LossHistory()
model.fit(x_train,y_train, epochs=500, steps_per_epoch=1)
model.save('ColorModel.h5')

# Validate the Model

In [ ]:
y_hat = model.predict(x_test)
score = 0
for i in range(len(y_hat)):
    score += (np.argmax(y_hat[i]) == np.argmax(np.array(y_test)[i]))
score / len(y_hat)

# CoreML
Convert to a CoreML Model using CoreMLTools

In [ ]:
cmmodel = cmt.converters.convert('ColorModel.h5', input_names='RGB', output_names='probs')
cmmodel.save('ColorModel.mlmodel')